In [1]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile
from ahocorasick import Automaton
from tqdm.auto import tqdm

/Users/dov/miniconda3/envs/cloud/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Code to clean the data

In [2]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
csv_file = zip_file.namelist()[0]
data = pd.read_csv(zip_file.open(csv_file))

/var/folders/jb/ntxm50wx2wv90pnq36yd8w940000gn/T/ipykernel_40406/804822752.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open(csv_file))


In [3]:
# data.iloc[0]

Removing useless fields

In [4]:
df = data.drop(["claimed", 
           "awards", 
           "popularity_detailed", 
           "popularity_generic", 
           "price_range", 
           "features", 
           "open_days_per_week", 
           "open_hours_per_week", 
           "working_shifts_per_week",
           "default_language",
           "reviews_count_in_default_language",
           "excellent",
           "very_good",
           "average",
           "poor",
           "terrible",
           "food",
           "service",
           "value",
           "atmosphere",
           "keywords"], axis=1)

Populating the "City" field taking it from the address

In [5]:
comuni_df = pd.read_csv("./elenco_comuni_italiani.csv", encoding='iso-8859-1', delimiter=";")

In [6]:
#TODO: "Rome" non viene preso, quindi risolvi questa cosa
def build_automaton(keywords):
    automaton = Automaton()
    for keyword in keywords:
        automaton.add_word(keyword, keyword)
    automaton.make_automaton()
    return automaton

keywords = [row['Denominazione in italiano'].lower() for _, row in comuni_df.iterrows() if type(row["Denominazione in italiano"]) == str]
automaton = build_automaton(keywords)

def find_comune(address):
    found = set()
    for item in automaton.iter(",".join(address.split(",")[1:]).lower()):
        found.add(item[1])
    if found:
        return max(found, key=len)
    else:
        return None

tqdm.pandas()
df['city'] = df['address'].progress_apply(find_comune)



100%|██████████| 224763/224763 [00:00<00:00, 939201.41it/s]


In [7]:
final_df = df.copy()
final_df['total_reviews_count'] = final_df['total_reviews_count'].fillna(0.0)
final_df['avg_rating'] = final_df['avg_rating'].fillna(0.0)
final_df = final_df.fillna("")
final_df = final_df[final_df["latitude"] != ""]
final_df = final_df[final_df["longitude"] != ""]

In [8]:
df = final_df.reset_index()

### Save the dataframe as a zipped file

In [9]:
# from zipfile import ZipFile
# import os

# DF_PATH = "./italian_restaurant_clean.csv"
# ZIP_PATH = "./italian_restaurant_clean.zip"
# final_df.to_csv(DF_PATH)
# with ZipFile(ZIP_PATH, "w") as myzip:
#     myzip.write(DF_PATH)

# os.remove(DF_PATH)

# Code to insert the restaurants in the database 

In [10]:
# df = pd.read_csv("./italian_restaurant_clean.zip")

In [11]:
# import tensorflow as tf
# import tensorflow_hub as hub
# import os
# os.environ['TFHUB_DOWNLOAD_PROGRESS'] = "1"

# # Load the Universal Sentence Encoder
# model = hub.load(
#                 "https://tfhub.dev/google/universal-sentence-encoder-large/5")
# def embed_name(name):
#     embeddings = model([name])
#     return embeddings[0]

In [12]:
# restaurant1 = "Mariuccio"
# restaurant2 = "Mariuccio"

# # Embed the restaurant names
# embedding1 = embed_name(restaurant1)
# embedding2 = embed_name(restaurant2)

# # Compute similarity between the embeddings
# distance = 1 - np.inner(embedding1, embedding2)

# print(distance)

In [13]:
# from minio import Minio
BASE = "http://proxy-load-balancer-203641034.us-east-1.elb.amazonaws.com"
URL = f'{BASE}/restaurants/create-restaurant'

# client = Minio(
#     "localhost:9002",
#     access_key="Dec6s4zuTzoVhnvYmALO",
#     secret_key="cWizD0DDGjGPNfflDDDOXmjE9VvzdEXvQrOZPEpI",
#     secure=False  # Otherwise bug
# )

Define the function that gets the geographical zone from the coordinates

In [14]:
import requests

def getGeoZoneFromLatLng(coordinates):
    latitude, longitude = coordinates['latitude'], coordinates['longitude']
    URL = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}"

    response = requests.get(URL)
    data = response.json()
    address = data['address']

    if 'quarter' in address and 'Municipio' in address['quarter']:
        print('quarter:', address['quarter'])
        return address['quarter']
    elif 'highway' in address and 'Municipio' in address['highway']:
        print('highway:', address['highway'])
        return address['highway']
    elif 'village' in address and 'Municipio' in address['village']:
        print('village:', address['village'])
        return address['village']
    elif 'neighbourhood' in address and 'Municipio' in address['neighbourhood']:
        print('neighbourhood:', address['neighbourhood'])
        return address['neighbourhood']
    elif 'town' in address and 'Municipio' in address['town']:
        print('town:', address['town'])
        return address['town']
    elif 'hamlet' in address and 'Municipio' in address['hamlet']:
        print('hamlet:', address['hamlet'])
        return address['hamlet']
    elif 'railway' in address and 'Municipio' in address['railway']:
        print('railway:', address['railway'])
        return address['hamlet']
    elif 'suburb' in address:
        if 'Municipio' in address['suburb']:
            print('suburb:', address['suburb'])
            print(address)
        else:
            return address['suburb']
    elif 'road' in address:
        if 'Municipio' in address['road']:
            print('road:', address['road'])
        else:
            return address['road']
    else:
        return ''

response = getGeoZoneFromLatLng({"latitude": 41.909734, "longitude": 12.349999})
print(response)

suburb: Municipio Roma XIII
{'road': 'Piazza Don Gustavo Cece', 'suburb': 'Municipio Roma XIII', 'village': 'Pantan Monastero', 'city': 'Roma', 'county': 'Roma Capitale', 'ISO3166-2-lvl6': 'IT-RM', 'state': 'Lazio', 'ISO3166-2-lvl4': 'IT-62', 'country': 'Italia', 'country_code': 'it'}
None


Code to upload restaurants in batches

In [15]:
import requests
from tqdm import tqdm
import json

# BASE = "http://localhost:3002"
# URL = f'{BASE}/create-restaurant-batch'

BASE = "http://proxy-load-balancer-203641034.us-east-1.elb.amazonaws.com"
URL = f'{BASE}/restaurants/create-restaurant-batch'

num_images = 6438

def assign_image(index):
    return index % num_images


# Modify the create_restaurant function to accept a DataFrame slice
def create_restaurants(df_slice):
    body = []

    for _, row in df_slice.iterrows():
        try:
            restaurant = {
                "name": row["restaurant_name"],
                "embeddingName": f"embedding_{(row['index'])}",
                "imageName": f"restaurant_image_{assign_image(row['index'])}",
                "address": row["address"],
                "latitude": float(row["latitude"]),
                "longitude": float(row["longitude"]),
                "country": row["country"],
                "region": row["region"],
                "province": row["province"],
                "city": row["city"],
                "tags": row["top_tags"],
                "cuisines": row["cuisines"],
                "specialDiets": row["special_diets"],
                "priceLevel": row["price_level"],
                "meals": row["meals"],
                "avgRating": float(row["avg_rating"]),
                "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
                "veganFriendly": row["vegan_options"] == "Y",
                "glutenFree": row["gluten_free"] == "Y",
                "reviewsNumber": int(row["total_reviews_count"])
            }
            body.append(restaurant)

            zone = getGeoZoneFromLatLng({"latitude": float(row["latitude"]), "longitude": float(row["longitude"])})
            restaurant["zone"] = zone
            print(zone, restaurant["address"])
        except requests.HTTPError as error:
            print("Error:", error)
            

    # requests.post(URL, json=body)

# Split the DataFrame into batches of restaurants
batch_size = 100
num_batches = len(df) // batch_size + 1
for i in tqdm(range(num_batches)):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, len(df))
    df_slice = df[start_index:end_index]

    create_restaurants(df_slice)

  0%|          | 0/2227 [00:00<?, ?it/s]

Via Jacopo Ortis Via Jacopo Ortis 15, 33090, Vito d'Asio Italy
Via Chiamp Via Chiamp, 18, 33090, Vito d'Asio Italy
Via Galantz Via Galantz 2, 33090 San Francesco, Vito d'Asio Italy
Dorfstraße - Via del Borgo Via del Borgo 4B, 39030 Valdaora di Sopra, Valdaora Italy
Piazza della Libertà Piazza Liberta' 14, 40060 Toscanella, Dozza Italy
Via Salvador Allende Via Allende Parco Ruggi, 40060 Toscanella, Dozza Italy
Piazza Antonio Gramsci Piazza Antonio Gramsci 2, 40060 Toscanella, Dozza Italy
Via XXIX Settembre Piazza Liberta' 10, 40060 Toscanella, Dozza Italy
error
Via Pietro Nenni Via Pietro Nenni 1, 40060 Toscanella, Dozza Italy
Centro storico Via della Roggia Grande 22, 38122 Stelvio Italy
Sentiero del Lago d'Oro Stilfser Joch, 39020 Stelvio Italy
Rosimstraße - Via Rosim Localita Solda' 59, 39029 Stelvio Italy
 Passo Stelvio, 39029 Stelvio Italy
Gomagoi Gomagoi 18, 39029 Stelvio Italy
LS/SP6 Stilfs, 39029 Stelvio Italy
 Contrada Santa Maria Casalpiano, 86040 Morrone del Sannio Italy
Via 

Via Cristoforo Colombo SP36, 97017, Santa Croce Camerina, Sicily Italy


In [9]:
# def get_num_images():
#     objects = client.list_objects(bucket_name="images")
#     num_images = sum(1 for _ in objects)
#     return num_images

# num_images = 6438

# def assign_image(index):
#     return index % num_images

# REMOVABLE_WORDS = ["gelateria", "pizzeria", "trattoria", "ristorante", "bisteccheria"]
# def clean_name(name):
#     return " ".join([word for word in name.lower().split(" ") if word not in REMOVABLE_WORDS])

# def create_restaurant(row):
#     body = {
#             "name": row["restaurant_name"],
#             "embeddingName": f"embedding_{(row['index'])}",
#             "imageName": f"restaurant_image_{assign_image(row['index'])}",
#             "address": row["address"],
#             "latitude": float(row["latitude"]),
#             "longitude": float(row["longitude"]),
#             "country": row["country"],
#             "region": row["region"],
#             "province": row["province"],
#             "city": row["city"],
#             "tags": row["top_tags"],
#             "cuisines": row["cuisines"],
#             "specialDiets": row["special_diets"],
#             "priceLevel": row["price_level"],
#             "meals": row["meals"],
#             "avgRating": float(row["avg_rating"]),
#             "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
#             "veganFriendly": row["vegan_options"] == "Y",
#             "glutenFree": row["gluten_free"] == "Y",
#             "reviewsNumber": int(row["total_reviews_count"])
#         }

    
#     result = requests.post(URL, json=body)

# import json
# import io
# def insert_embeddings(row):
#     BUCKET_NAME = "embeddings"
#     embedding =  embed_name(clean_name(row["restaurant_name"])).numpy().tolist()
#     embedding_str = json.dumps(embedding)
#     embedding_bytes = embedding_str.encode("utf-8")
#     client.put_object(
#             BUCKET_NAME, f"embedding_{row['index']}", io.BytesIO(embedding_bytes), length=len(embedding_bytes))

# tqdm.pandas()
# # df.progress_apply(create_restaurant, axis=1)
# # df.progress_apply(insert_embeddings, axis=1)

  1%|          | 1485/222603 [09:02<22:27:15,  2.74it/s]


KeyboardInterrupt: 